# Technical Interview

This document presents notes and codes about 3 exercices. The code will be in python 2, the data is given in a separate csv (too big for github).

The data were extract from two compressed files. The files were corrupt so I had to use `bzip2recover` to uncompress them.

## Exercise 1

### Task: count the number of lines in Python for each file

We use a csv reader to iterate over the whole file, using a small counter to count the number of (not empty) lines.
Reading the whole file with `f.read()` won't work because of the size of the files.

In [ ]:
import csv

with open("./data/searches.csv","r") as f:
    reader = csv.reader(f,delimiter="^")
    i = 0
    for row in reader:
        if row!="\n" and row!="":
            i +=1
    print "nb_searches = %d"%i
with open("./data/bookings.csv","r") as f:
    reader = csv.reader(f,delimiter="^")
    i = 0
    for row in reader:
        if row!="\n" and row!="":
            i +=1
    print "nb_bookings = %d"%i